#Parte responsável por treinar o reconhecimento das mão

In [1]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

2023-05-16 10:14:40.783973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
RANDOM_SEED = 42
NUM_CLASSES = 3

In [3]:
dataset = "model/keypoint_classifier/keypoint.csv"
model_save_path = "model/keypoint_classifier/keypoint_classifier.hdf5"

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=",", dtype="float32", usecols=list(range(1, 43)))
Y_dataset = np.loadtxt(dataset, delimiter=",", dtype="int32", usecols=(0))

X_train, X_test, Y_train, Y_test = train_test_split(X_dataset, Y_dataset, train_size=0.8, random_state=RANDOM_SEED)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((42, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

2023-05-16 10:15:04.717156: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [7]:
callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_best_only=False
)

abort_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [8]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

#Compilamento de rede neural


In [9]:
model.fit(
    X_train,
    Y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, Y_test),
    callbacks=[callback, abort_callback]
)

#Treinamento efetivo

Epoch 1/1000
28/28 [==============================] - ETA: 0s - loss: 1.1222 - accuracy: 0.3435
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
28/28 [==============================] - 4s 58ms/step - loss: 1.1222 - accuracy: 0.3435 - val_loss: 1.0746 - val_accuracy: 0.4303
Epoch 2/1000
24/28 [========================>.....] - ETA: 0s - loss: 1.0715 - accuracy: 0.4131
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
28/28 [==============================] - 1s 31ms/step - loss: 1.0687 - accuracy: 0.4138 - val_loss: 1.0308 - val_accuracy: 0.6303
Epoch 3/1000
28/28 [==============================] - ETA: 0s - loss: 1.0400 - accuracy: 0.4461
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
28/28 [==============================] - 1s 47ms/step - loss: 1.0400 - accuracy: 0.4461 - val_loss: 0.9866 - val_accuracy: 0.6697
Epoch 4/1000
27/28 [===========================>..] - ETA: 0s - loss: 1.0097 - accuracy: 0.488

In [10]:
val_loss, val_acc = model.evaluate(X_test, Y_test, batch_size=128)

7/7 [==============================] - 0s 13ms/step - loss: 0.1658 - accuracy: 0.9708


In [11]:
model = tf.keras.models.load_model(model_save_path)

In [12]:
result_predict = model.predict(np.array([X_test[0]]))
print(np.squeeze(result_predict))

1/1 [==============================] - 0s 305ms/step
[0.8522926  0.10801317 0.03969432]


In [13]:
model.save(model_save_path, include_optimizer=False)

In [14]:
#quantização
tfl_path = "model/keypoint_classifier/keypoint_classifier.tflite"

convertio = tf.lite.TFLiteConverter.from_keras_model(model=model)
convertio.optimizations = [tf.lite.Optimize.DEFAULT]
tfl_quantized_model = convertio.convert()

open(tfl_path, "wb").write(tfl_quantized_model)


2023-05-16 10:18:55.496833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-05-16 10:18:55.672123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,42]
	 [[{{node inputs}}]]
2023-05-16 10:18:55.711088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-05-16 10:1

INFO:tensorflow:Assets written to: /tmp/tmpp160fmx4/assets


INFO:tensorflow:Assets written to: /tmp/tmpp160fmx4/assets
2023-05-16 10:18:58.429797: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-16 10:18:58.429845: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-16 10:18:58.430460: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpp160fmx4
2023-05-16 10:18:58.432702: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 10:18:58.432738: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpp160fmx4
2023-05-16 10:18:58.434296: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-05-16 10:18:58.442393: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is n

6500

In [15]:
interpreter = tf.lite.Interpreter(model_path=tfl_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]["index"], np.array([X_test[0]]))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [16]:
%%time
interpreter.invoke()
tfl_results = interpreter.get_tensor(output_details[0]["index"])


CPU times: user 1.48 ms, sys: 0 ns, total: 1.48 ms
Wall time: 15.1 ms
